In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Charger les données
# Supposons que data contient les données textuelles et labels les ICD10
data=pd.read_csv('data.csv')
# Prétraitement des données textuelles

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['RawText'])
sequences = tokenizer.texts_to_sequences(data['RawText'])
sequences = pad_sequences(sequences, maxlen=100)

# Encodage des labels ICD10
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data[' ICD10'])

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Construction du modèle
model = Sequential()
model.add(Embedding(input_dim=50000, output_dim=300, input_length=100))
model.add(Bidirectional(GRU(256)))
model.add(Dense(700, activation='relu'))
model.add(Dense(14602, activation='softmax'))  # Assure-toi que le nombre de neurones correspond au nombre de classes ICD10

# Compilation du modèle
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraînement du modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
model.save("THEModel.h5")

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Perte sur l'ensemble de test:", loss)
print("Précision sur l'ensemble de test:", accuracy)

In [ ]:
def predire_code_icd():
    # Récupérer le texte d'entrée de l'utilisateur
    texte_entrée = input("Entrez votre texte ici : ")

    # Procéder à la tokenisation et au padding
    sequence_entrée = tokenizer.texts_to_sequences([texte_entrée])
    sequence_padded = pad_sequences(sequence_entrée, maxlen=100)  # Assurez-vous que cela correspond à la longueur utilisée lors de l'entraînement

    # Faire une prédiction avec le modèle
    prediction = model.predict(sequence_padded)

    # Obtenir l'indice de la classe la plus probable
    indice_classe_predite = np.argmax(prediction, axis=-1)

    # Convertir l'indice en code ICD
    code_icd_predite = label_encoder.inverse_transform(indice_classe_predite)[0]

    # Retourner ou imprimer le code ICD prédit
    print(f"Code ICD prédit: {code_icd_predite}")

# Appeler la fonction pour faire une prédiction basée sur l'entrée de l'utilisateur
predire_code_icd()